In [1]:
!nvidia-smi

Thu Oct  3 08:18:58 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   30C    P0              42W / 400W |      2MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [2]:
path = '/content/drive/MyDrive/DACON/Finance/reprocessed/'
path = '/content/drive/MyDrive/kdt-EST-AI/project/dacon_fis/src/'
base_directory = path # Your Base Directory

In [3]:
dataset_name = "kdt3/DACON-QA-large-ensemble-markdown-reprocessed-256"
train_name = "kdt3/DACON-QA-large-ensemble-markdown-refactored-256"
fname = "gemma2_large_ensemble_markdown_256_3epoch_refactored.csv"

# 설명

## Question - Answering with Retrieval

본 대회의 과제는 중앙정부 재정 정보에 대한 **검색 기능**을 개선하고 활용도를 높이는 질의응답 알고리즘을 개발하는 것입니다. <br>이를 통해 방대한 재정 데이터를 일반 국민과 전문가 모두가 쉽게 접근하고 활용할 수 있도록 하는 것이 목표입니다. <br><br>
베이스라인에서는 평가 데이터셋만을 활용하여 source pdf 마다 Vector DB를 구축한 뒤 langchain 라이브러리와 llama-2-ko-7b 모델을 사용하여 RAG 프로세스를 통해 추론하는 과정을 담고 있습니다. <br>( train_set을 활용한 훈련 과정은 포함하지 않으며, test_set  에 대한 추론만 진행합니다. )

## Mount/Login

구글 드라이브를 마운트하고 허깅페이스에 로그인
- 이때 허깅페이스 토큰은 kdt3 그룹에 대해 읽기/쓰기 권한이 있는 토큰이어야 함

## Download Library
필요/사용 라이브러리 다운로드
이때 버전 문제로 설치를 한 뒤 세션을 한번 재시작해줘야 합니다
<br>(그리고 세션 완전히 끊기면 다운로드 후 재시작을 다시 해줘야...)

## Import Library
한번 재시작했으면 위 과정 없이 Import만 실행해주면 됩니다

## Vector DB
문서를 여러 조각(chunk)로 나누고, 임베딩 유사도를 통해 관련 조각을 찾을 수 있게 DB화하는 함수들이 정의되어 있습니다.

## DB 생성
Vector DB에서 정의된 함수들로 문서 DB를 만들어줍니다.<br><br>
이때 Train과 Test를 한번에 하려고 하면 코랩이 터질 확률이 높으므로 Train하고 Create Dataset까지 실행해 업로드 한 뒤 재시작해서 램을 비우고 Test를 하는 것이 좋습니다.<br> 또한 문서 임베딩을 어떤 모델로 할지 인자로 넘겨줄 수 있습니다

## Create Dataset
DB 생성에서 만든 db와 데이터 dataframe을 사용해 HuggingFace 데이터셋 생성 후 업로드

## Fine-Tuning
학습 데이터셋으로 모델에 대한 파인튜닝 진행 후 Huggingface에 업로드<br>
4비트 양자화 LoRA로 파인튜닝<br>
기반 모델 또는 넣어줄때 사용할 프롬프트, 학습 관련 하이퍼파라미터 수정 가능

## Langchain 을 이용한 추론
모델을 사용한 추론


## 실행
### 기본
Mount/Login -> Download Library -> 재시작 (처음 1번)
Mount/Login -> Import Library (이후)

### 데이터셋 만들기
기본 -> Vector DB -> DB 생성 -> Create Dataset에서 첫 셀 + Train/Valid/Test 중 해당하는 셀

### 모델 학습하기
기본 -> Fine-Tuning(업로드할 위치, 데이터셋 위치, 모델 링크 확인 필수)

### 학습된 모델로 추론하기
기본 -> Langchain을 이용한 추론(모델 링크, 데이터셋 위치 확인) -> Submission(저장할 파일명 확인)

# Mount/Login

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
import os

token_path = os.path.join(base_directory,'data','token')
with open(token_path,'r') as f:
    hf_token = f.readline().strip('\n')
    wandb_token = f.readline().strip('\n')

In [6]:
from huggingface_hub import login

login(token=hf_token, add_to_git_credential=True)

Token is valid (permission: fineGrained).
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [7]:
!pip install wandb
import wandb

# wandb 개인 API 키 입력
wandb.login(key=wandb_token)

import os

os.environ["WANDB_PROJECT"]="DACON_FinAI"
os.environ["WANDB_LOG_MODEL"] = "checkpoint"
wandb_run_name="noaug-Llama3.1-lora-base"

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin


# Download Library

In [8]:
!apt-get install tesseract-ocr
!apt-get install poppler-utils

!pip install orjson==3.10.6

!pip install accelerate
!pip install -i https://pypi.org/simple/ bitsandbytes
!pip install transformers[torch] -U

!pip install datasets
!pip install langchain
!pip install langchain_community
!pip install langchain-teddynote
!pip install sentence-transformers
!pip install faiss-gpu
!pip install peft
!pip install trl

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
poppler-utils is already the newest version (22.02.0-2ubuntu0.5).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
Looking in indexes: https://pypi.org/simple/


# Import Library

In [9]:
import os
import unicodedata

import torch
import pandas as pd
from tqdm.auto import tqdm

from langchain.document_loaders.parsers.pdf import PDFPlumberParser


from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    pipeline,
    BitsAndBytesConfig
)
from accelerate import Accelerator

# peft
from peft import prepare_model_for_kbit_training
from peft import PeftModel
from peft import LoraConfig, get_peft_model


# Langchain 관련
from langchain.llms import HuggingFacePipeline
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter, MarkdownHeaderTextSplitter
from langchain.prompts import PromptTemplate
from langchain.schema.runnable import RunnablePassthrough, RunnableParallel
from langchain.schema.output_parser import StrOutputParser

# PDF 로딩/청크화 관련
from langchain.document_loaders.parsers.pdf import PDFPlumberParser
from langchain.document_loaders.pdf import PDFPlumberLoader
from langchain.document_loaders import UnstructuredPDFLoader
from langchain_teddynote.retrievers import KiwiBM25Retriever
from langchain.retrievers import EnsembleRetriever, MultiQueryRetriever


/usr/local/lib/python3.10/dist-packages/langchain_teddynote/retrievers/__init__.py:14: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  from .kiwi_bm25 import KiwiBM25Retriever


# Fine-Tuning

In [11]:
# 모델 ID
model_cands={
 'llama2' : "beomi/llama-2-ko-7b",
 'yi' : "beomi/Yi-Ko-6B",
 'solar-beom' : "beomi/Solar-Ko-Recovery-11B",
 'gemma2' : "rtzr/ko-gemma-2-9b-it",
 'solar-lee' : "chihoonlee10/T3Q-ko-solar-dpo-v8.0",
 'llama31' : "KISTI-KONI/KONI-Llama3-8B-Instruct-20240729",
}

model_id = model_cands['gemma2']

In [12]:
# 모델 로드 및 양자화 설정 적용

def load_model_w_setting(model_id,add_output_token=False,**kwargs):
  # 4비트 양자화 설정
  bnb_config = BitsAndBytesConfig(
      load_in_4bit=True,
      bnb_4bit_use_double_quant=True,
      bnb_4bit_quant_type="nf4",
      bnb_4bit_compute_dtype=torch.bfloat16
  )

  # 토크나이저 로드 및 설정
  tokenizer = AutoTokenizer.from_pretrained(model_id)
  tokenizer.use_default_system_prompt = False
  tokenizer.padding_side="right"

  model= AutoModelForCausalLM.from_pretrained(
      model_id,
      quantization_config=bnb_config,
      device_map="auto",
      trust_remote_code=True,
      **kwargs
      )

# 일부 모델의 경우 토크나이저에 답변 토큰 추가 작업 필요
  if add_output_token :
    initial_token_count = len(tokenizer)
    response_template = '답변: '
    added_token_count = tokenizer.add_special_tokens({"additional_special_tokens": [response_template]})
    model.resize_token_embeddings(new_num_tokens=initial_token_count+added_token_count)

  return model, tokenizer

In [13]:
model,tokenizer = load_model_w_setting(model_id,attn_implementation='eager')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/40.5k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

...

model-00008-of-00010.safetensors:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

model-00009-of-00010.safetensors:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

model-00010-of-00010.safetensors:   0%|          | 0.00/705M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/10 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

In [14]:
#데이터셋 로드
from datasets import load_dataset

# dataset_url = "kdt3/DACON-QA-base-table-preprocessed-v3"
# dataset_url = "kdt3/DACON-QA-base-preprocessed-v2"
# dataset_url = "kdt3/DACON-QA-base-augselect"
# dataset_url = "kdt3/DACON-QA-base-markdown"
# dataset_url = "kdt3/DACON-QA-bge-markdown"
dataset_url = dataset_name


train_dataset = load_dataset(dataset_url)

README.md:   0%|          | 0.00/543 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/583k [00:00<?, ?B/s]

valid-00000-of-00001.parquet:   0%|          | 0.00/156k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/104k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/496 [00:00<?, ? examples/s]

Generating valid split:   0%|          | 0/100 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/98 [00:00<?, ? examples/s]

In [15]:
# 문맥 잘못됐나 확인 - 문맥 길이 체크
import numpy as np
amax = np.argmax([len(x) for x in train_dataset['train']['context']])
amax, len(train_dataset['train']['context'][amax])

(77, 1955)

In [16]:
def print_dataset_ele(data,i):
  print(data['question'][i])
  print('--------')
  print(data['context'][i])
  print('--------')
  print(data['answer'][i])

#예시
i= amax
print_dataset_ele(train_dataset['train'],i)

예산 증감률이 가장 높은 분야는 무엇인가요?
--------
예산개요  
-----  
y g  
제
1  
##### 보건 · 복지 · 고용 분야 부  
## 01  
1-1 재정지원 방향  
2024년 보건·복지·고용분야 재정투자 규모는 242.9조원 수준이며, 총지출 대비 비중이  
37.0%로 12대 분야 중 가장 비중이 크다. 소득·고용·주거 안전망에 대한 투자  
2024년도 예산안 시정연설 및 제안설명  
확충으로 저소득·노인·장애인 등 취약계층을 더욱 두텁게 지원하고, 가족돌봄청년 등
0.9%이다.  
[표 3-2] 2024년 예산지출 규모  
(단위: 조원, %)  
|(nan, '구 (nan, '202|3년 (nan, '|2024년 (nan, '|증감률 \n(B-|
|---|---|---|---|  
분') **\n(A)')** **\n(B)')** **A)/A')**  
0 ■ 예산지출 441.1 438.3 △0.6  
1 •일반회계 369.4 356.5 △3.5  
2 •특별회계 71.6 81.7 14.1
근무복 점퍼도 보온력이 높은 구스 재질로 고품질화하는 한편, 시설관리, 청사방호,  
견위생, 통역 등 공무직 인원 증원도 추진할 예정이다.  
-----  
0 Summary of Budget for FY 2024  
[표 11-4] 경찰 부문 주요 변동내역  
(단위: 억원, %)  
0 합 계 128,064 133,486 4.2  
|구 분 2 (|023년 2 a) (|024년 b)|증감률 ((b-a|)/ 비 고 a)|
계획이다.  
-----  
0 2024년 나라살림예산개  
0 2024  
[표 6-5] 항공·공항 부문 주요 변동내역  
|구 분 2023년 (|b) 2024년 (b|) 증감률 ((b-|a)/a)|
|---|---|---|---|  
1 공항건설 및 관리 1,601 7,374 360.5  
2 항공발전, 운영지원 1,834 1,532 △16.4  
|Col1|Col2|Col3|Col4|
|---|---|---|-

In [17]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=16,
    lora_alpha=64,
    target_modules=[
        "q_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
        "lm_head",
    ],
    bias="none",
    lora_dropout=0.05,
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, config)

/usr/local/lib/python3.10/dist-packages/peft/tuners/tuners_utils.py:498: UserWarning: Model with `tie_word_embeddings=True` and the tied_target_modules=['lm_head'] are part of the adapter. This can lead to complications, for example when merging the adapter or converting your model to formats other than safetensors. See for example https://github.com/huggingface/peft/issues/2018.
  warnings.warn(


In [47]:
# 학습 프롬프트 - 추론 프롬프트와 통일하는 것이 좋다고 함
template = '''
"task_instructions" : [

 당신은 재정 정보 관련 전문가 입니다. 문서를 바탕으로 질문에 한 문장 이내로 답변하세요.
 1. 문서에 있는 내용을 자르거나 편집하지 않고 그대로 가져오세요.
 2. 순서에 따른 번호를 매기지 마세요. 출력 시 불이익을 줄 것입니다.
 3. 수치에 단위가 있다면 문서를 바탕으로 답변에 단위를 포함하세요.
 4. 질문의 키워드를 바탕으로 문서를 끝까지 검토하세요.
 5. 한 단어 혹은 단어의 나열이 아닌, 완성된 한국어 문장으로 답변하세요.
 6. 답변 외에 예시, 참고, 정보 출처, 신뢰도, 확장된 답변, '답변: ', '참고: '를 절대로 출력하지 마세요.

]

"context":
{context},

"question":
{question},

"주어진 질문에 대한 답변만 한 문장으로 생성한다."

"answer":
{answer}'''

response_template = '"answer":\n'

def formatting_prompts_func(example, template=template):
    output_texts = []
    for i in range(len(example['question'])):
        context = example['context'][i]
        question = example['question'][i]
        answer = example['answer'][i]
        output_texts.append(template.format(context=context,question=question,answer=answer))
    return output_texts



In [19]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score
import collections
# from SQUAD, 메트릭 계산 함수

def compute_f1(y_true, y_pred):
    common = collections.Counter(y_true) & collections.Counter(y_pred)
    num_same = sum(common.values())
    precision = 1.0 * num_same / len(y_pred) if len(y_pred) != 0 else 0
    recall = 1.0 * num_same / len(y_true) if len(y_true) != 0 else 0
    if len(y_pred) == 0 or len(y_true) == 0: f1 = int(y_pred == y_true)
    elif num_same == 0: f1 = 0
    else: f1 = (2 * precision * recall) / (precision + recall)
    return f1, precision, recall

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1, precision, recall = compute_f1(labels,preds)
#    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    auc = roc_auc_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall,
        'auroc': auc
    }

In [50]:
import transformers, os
from datetime import datetime
from trl import SFTConfig, SFTTrainer, DataCollatorForCompletionOnlyLM

# 로컬에 모델 저장하고 싶은 경우 이름 지정
project = "financeQA-finetune"
base_model_name = "gemma2"
run_name = base_model_name + "_" + project
output_dir = os.path.join(path,run_name)
if not os.path.exists(output_dir) : os.makedirs(output_dir)

MAX_LEN = 2048

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=train_dataset['train'],
    eval_dataset = train_dataset['valid'],
    compute_metrics=compute_metrics,
    args=transformers.TrainingArguments(
        do_eval=True,
        output_dir=output_dir,
        warmup_ratio=0.05,
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        gradient_checkpointing=True,
        num_train_epochs = 3,
        learning_rate=2e-4,
        fp16=True,
        optim="paged_adamw_8bit",
        logging_strategy='epoch',
        report_to="wandb",
        run_name=wandb_run_name,
        # save_strategy="steps",
        # save_steps=25,

    ),
    max_seq_length=MAX_LEN,
    formatting_func=formatting_prompts_func, # 프롬프트 처리하기 위해 필요
    data_collator=DataCollatorForCompletionOnlyLM(response_template, tokenizer=tokenizer, mlm=False), #모델에게 답변생성에만 집중하도록 함
)

model.config.use_cache = False
trainer.train()

trained_model = (trainer.model.module if hasattr(trainer.model, "module") else trainer.model)

#로컬에 저장할 경우
# trained_model.save_pretrained(f"{output_dir}/saved_model")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': max_seq_length. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:283: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


Map:   0%|          | 0/496 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:92: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


Step,Training Loss
124,0.693100
248,0.662500
372,0.644000


/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:227: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")
wandb: Adding directory to artifact (/content/drive/MyDrive/kdt-EST-AI/project/dacon_fis/src/gemma2_financeQA-finetune/checkpoint-372)... Done. 6.7s
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:227: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")


In [51]:
train_name

'kdt3/DACON-QA-large-ensemble-markdown-refactored-256'

In [54]:
# 모델 업로드
trained_model.push_to_hub(train_name+'-epoch6', private=True,save_embedding_layers=True)

adapter_model.safetensors:   0%|          | 0.00/2.05G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/kdt3/DACON-QA-large-ensemble-markdown-refactored-256-epoch6/commit/f2ab3d1c90c4e7f570356ae51ec0d4a4d53313be', commit_message='Upload model', commit_description='', oid='f2ab3d1c90c4e7f570356ae51ec0d4a4d53313be', pr_url=None, pr_revision=None, pr_num=None)

# Langchain 을 이용한 추론

In [63]:
# gpu memory 할당 해제
import gc, time

def free_cuda():
  mem = 1
  while mem > 0 :
    time.sleep(0.5)
    torch.cuda.empty_cache()
    mem = gc.collect()
    print("freed : ",mem)

free_cuda()

freed :  0


In [64]:
# HuggingFacePipeline 객체 생성

# 모델 ID
model_id = model_cands[run_name.split('_')[0]]
peft_model_id = train_name
trained_model,tokenizer = load_model_w_setting(model_id)

#Fine-Tune 한 LoRA 어댑터 불러오기
trained_model.load_adapter(peft_model_id)

text_generation_pipeline = pipeline(
    model=trained_model,
    tokenizer=tokenizer,
    task="text-generation",
    return_full_text=False,
    max_new_tokens=200,
    # repetition_penalty=1.5,
    eos_token_id = tokenizer.eos_token_id,
    pad_token_id = tokenizer.pad_token_id,
    max_length=MAX_LEN
)

llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

Loading checkpoint shards:   0%|          | 0/10 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/peft/tuners/tuners_utils.py:498: UserWarning: Model with `tie_word_embeddings=True` and the tied_target_modules=['lm_head'] are part of the adapter. This can lead to complications, for example when merging the adapter or converting your model to formats other than safetensors. See for example https://github.com/huggingface/peft/issues/2018.
  warnings.warn(


In [65]:
#데이터셋 로드
from datasets import load_dataset
dataset_url = dataset_name
dataset = load_dataset(dataset_url)

In [66]:
# 검증 데이터 쓸지, 테스트 데이터 쓸지
#eval_mode = 'valid
eval_mode = 'test'
eval_dataset = dataset[eval_mode]

In [67]:
# 그냥 GPU 메모리 확인용
!nvidia-smi

Thu Oct  3 10:02:58 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   30C    P0              48W / 400W |  22071MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [68]:
template[:-8]

' \n"task_instructions" : [\n\n 당신은 재정 정보 관련 전문가 입니다. 문서를 바탕으로 질문에 한 문장 이내로 답변하세요.\n 1. 문서에 있는 내용을 자르거나 편집하지 않고 그대로 가져오세요.\n 2. 순서에 따른 번호를 매기지 마세요. 출력 시 불이익을 줄 것입니다.\n 3. 수치에 단위가 있다면 문서를 바탕으로 답변에 단위를 포함하세요.\n 4. 질문의 키워드를 바탕으로 문서를 끝까지 검토하세요.\n 5. 한 단어 혹은 단어의 나열이 아닌, 완성된 한국어 문장으로 답변하세요.\n 6. 답변 외에 예시, 참고, 정보 출처, 신뢰도, 확장된 답변, \'답변: \', \'참고: \'를 절대로 출력하지 마세요.\n\n]\n\n"context":\n{context},\n\n"question":\n{question},\n\n"주어진 질문에 대한 답변만 한 문장으로 생성한다."\n\n"answer":\n'

In [69]:
from tqdm.auto import trange

# 결과를 저장할 리스트 초기화
results = []

# DATASET 구조를 dataset[i]={'question':,'context':,...}로 바꾸면 안됨?

# Dataset 각 행에 대해 처리
for idx in trange(len(eval_dataset['question']), desc="Answering Questions"):
    #질문, 컨텍스트(문서)
    question = eval_dataset['question'][idx]
    context = eval_dataset['context'][idx]

    # RAG 체인 구성
    prompt = PromptTemplate.from_template(template[:-9])

    # RAG 체인 정의
    if context != "":
        rag_chain = (
            RunnableParallel(context=lambda x: x["context"], question = lambda x: x["question"])
            | prompt
            | llm
            | StrOutputParser()
        )
    else:
        rag_chain = (
            {"question": RunnablePassthrough()}
            | prompt
            | llm
            | StrOutputParser()
        )

    # 답변 추론
    print(f"Question: {question}")
    full_response = rag_chain.invoke({"question": question, "context": context})

    print(f"Answer: {full_response}\n")

    # 결과 저장

    if context != "":
        results.append({
            "Context": context,
            "Question": question,
            "Answer": full_response,
            "True_Answer": eval_dataset['answer'][idx]
        })
    else:
        results.append({
            "Question": question,
            "Answer": full_response,
            "True_Answer": eval_dataset['answer'][idx]
        })

Answering Questions:   0%|          | 0/98 [00:00<?, ?it/s]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Both `max_new_tokens` (=200) and `max_length`(=2048) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Question: 2022년 혁신창업사업화자금(융자)의 예산은 얼마인가요?


Both `max_new_tokens` (=200) and `max_length`(=2048) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Answer: 
2022년 혁신창업사업화자금(융자)의 예산은 2,330,000,000원입니다.


Question: 중소벤처기업부의 혁신창업사업화자금(융자) 사업목적은 무엇인가요?


...

Answer: 
미래사회보장급여에 대한 순의무는 현행법과 규정에 따라 이미 가입자가 획득한 미래 급여의 현재가치에서 현행법과 규정에 따른 사회보장제도의 미래보험료의 순현재가치를 차감한 것을 인식한다.


Question: 국가결산보고서와 지방자치단체 회계기준에서 우발부채에 대한 용어 및 회계처리가 어떻게 다른가요?


Both `max_new_tokens` (=200) and `max_length`(=2048) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Answer: 
국가결산보고서는 '우발사항'으로 공시하는 반면, 지방자치단체 회계기준은 '우발손실'로 정의하고 있습니다.


Question: 우발부채란 무엇이며, 그 관리가 왜 중요한가?


Both `max_new_tokens` (=200) and `max_length`(=2048) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Answer: 
우발부채는 미래의 다양한 의제의무를 포괄하는 재정위험을 발생시키는 구체적 원인으로, 재정위기나 예기치 못한 사건으로 인해 발생할 수 있으며, 지속가능한 재정운영을 위해 관리가 중요합니다.


Question: 보증이란 무엇이며, 어떤 형태의 보증이 재정상태표에 부채로 기록되는가? 또한 표준화 보증이란 무엇이며, 어떤 목적으로 발행되는가?
Answer: 
보증이란 피보증처의 부채에 대해 지급을 보증하는 것을 말하며, 파생금융상품 형태의 보증과 표준화 보증은 재정상태표에 부채로 기록되며, 표준화 보증은 통상 아주 적은 금액에 대해 획일적 조건으로 대규모로 발행하는 보증으로 수출, 환보증 등 다양한 목적으로 발행된다.




In [71]:
i = 28
print_dataset_ele(eval_dataset,i)

2023년 에너지바우처 사업 예산에서 사업운영비 중 시스템 고도화에 얼마가 할당되었나요?
--------
-  대상자 확대 등에 따른 홍보, 시스템 고도화, 콜센터 운영 비용 증액  
-  (산출) 에너지복지 홍보(448백만원), 시스템 고도화(705백만원), 콜센터 운영(502.7백만원), 패널조사  
(131.7백만원), 전달체계 구축(34.6백만원), 주택관리공단 운영지원(50백만원)  
ㅇ 2023년도 예산 및 2024년도 예산 산출 세부내역 비교
업운영비 (1,349백만원) ㆍ 에너지복 0.33만 세대 × 31만원 = 1,023백만원
지 홍보: 328백만원 ㆍ 시스템 고도 마. 사업운영비 (1,872백만원) ㆍ 에너지
화 : 520백만원 ㆍ 콜센터 운영 : 280 복지 홍보: 448백만원 ㆍ 시스템 고도화 :
백만원 ㆍ 패널조사 : 133.5백만원 705백만원 ㆍ 콜센터 운영 : 502.7백만원
ㆍ 에너지바우처 전달체계 구축 : ㆍ 패널조사 : 131.7백만원 ㆍ 에너지바우
-「새정부 에너지정책 방향(국무회의 의결)」을 통해 필수 에너지복지 확대[*] 발표(’22.7)  
-  (에너지바우처) 에너지바우처 지원 확대를 통해 에너지복지 사각지대 해소 추진  
-  2023년도 에너지바우처 사업 시행(‘23.5~’23.12월 신청·접수, ‘23.7~’24.4월 사용)  
□ 주요내용  
① 사업규모  
-  총사업비(해당되는 경우에만 기재) : 해당없음  
-  사업기간 : ’15년 ~ 계속
## 9) 최근 3년간 동 사업에 대한 주요 외부지적사항 및 평가, 문제점 및 대책  
□ ‘21년 국정감사 지적사항  
ㅇ 저소득층의 폭염 피해 최소화를 위해 동절기 에너지바우처 일부를 하절기에 사용  
할 수 있도록 제도 개선할 것  
- 희망세대 대상으로 동절기 에너지바우처 중 일부를 하절기에 사용할 수 있도록  
시스템 개선하였음(‘22.5월)  
□ ‘21년 결산 지적사항
백만원 라. 등유바우처 (1,023백만원) ㆍ 0.33만 세대 × 31만원 = 1,02

In [72]:
def post_process(ans):
  ans = ans.strip()
  return ans.split('\n')[0]

# Evaluation

In [73]:
rslt_df = pd.DataFrame(results)
rslt_df['Answer_post'] = rslt_df['Answer'].apply(post_process)

if eval_mode == 'valid':
  temp0 = rslt_df[['Answer','True_Answer']].apply(compute_f1,axis=1)
  temp1 = rslt_df[['Answer_post','True_Answer']].apply(compute_f1,axis=1)
  display(temp0)
  rslt_df[['F1_0','Precision_0','Recall_0']] = pd.DataFrame(temp0.tolist(), index= temp0.index)
  rslt_df[['F1_1','Precision_1','Recall_1']] = pd.DataFrame(temp1.tolist(), index= temp1.index)

rslt_df

,Context,Question,Answer,True_Answer,Answer_post
0,- 2022. 1 미래기술육성자금 및 고성장촉진자금 사업 폐지 \n- 2023...,2022년 혁신창업사업화자금(융자)의 예산은 얼마인가요?,"\n2022년 혁신창업사업화자금(융자)의 예산은 2,330,000,000원입니다.\n",,"2022년 혁신창업사업화자금(융자)의 예산은 2,330,000,000원입니다."
1,## 4. 사업목적 \nㅇ (창업기반지원) 기술력과 사업성이 우수하고 미래 성장가...,중소벤처기업부의 혁신창업사업화자금(융자) 사업목적은 무엇인가요?,\n중소벤처기업부의 혁신창업사업화자금(융자) 사업목적은 중소기업이 보유한 우수 기술...,,중소벤처기업부의 혁신창업사업화자금(융자) 사업목적은 중소기업이 보유한 우수 기술의 ...
2,|0 코드|5100 5152|301||\n|1 명칭|창업환경조성 창업기업지원|융자(...,중소벤처기업부의 혁신창업사업화자금(융자) 사업근거는 어떤 법률에 근거하고 있나요?,"\n중소기업진흥에 관한 법률 제66조, 제67조, 제74조, 중소기업창업지원법 제3...",,"중소기업진흥에 관한 법률 제66조, 제67조, 제74조, 중소기업창업지원법 제35조..."
3,"- 2014. 2 경제혁신 3개년 계획에 반영(청년전용창업자금, 재창업자금) \...",2010년에 신규 지원된 혁신창업사업화자금은 무엇인가요?,\n2010년에 재창업자금(실패 경영인에 대한 재기지원)이 신규 지원되었습니다.\n,,2010년에 재창업자금(실패 경영인에 대한 재기지원)이 신규 지원되었습니다.
4,"- 2014. 2 경제혁신 3개년 계획에 반영(청년전용창업자금, 재창업자금) \...",혁신창업사업화자금 중 2020년에 신규 지원된 자금은 무엇인가요?,\n2020년에는 미래기술육성자금과 고성장촉진자금이 신규로 지원되었습니다.\n,,2020년에는 미래기술육성자금과 고성장촉진자금이 신규로 지원되었습니다.
...,...,...,...,...,...
93,하는 공적보증채무(publicly guaranteed debt)에 해당 \n- ...,재정정책에서 공적보증채무와 다른 일회성 보증은 어떻게 구분되는가?,\n공적보증채무는 보증인이 기타 공공부문과 민간부문 단위 기존 채무의 원리금 상환을...,,공적보증채무는 보증인이 기타 공공부문과 민간부문 단위 기존 채무의 원리금 상환을 보...
94,"다고 여겨 정부의 계약상 부채를 초래하지 않는 것으로 간주하며, 우발부채로 인식 ...",미래사회보장급여에 대한 순의무란 무엇을 의미하는가?,\n미래사회보장급여에 대한 순의무는 현행법과 규정에 따라 이미 가입자가 획득한 미래...,,미래사회보장급여에 대한 순의무는 현행법과 규정에 따라 이미 가입자가 획득한 미래 급...
95,"- 「국가회계기준에 관한 규칙」은 제50조에서 ‘충당부채’, ‘우발부채’ 및 ‘우...",국가결산보고서와 지방자치단체 회계기준에서 우발부채에 대한 용어 및 회계처리가 어떻게...,"\n국가결산보고서는 '우발사항'으로 공시하는 반면, 지방자치단체 회계기준은 '우발손...",,"국가결산보고서는 '우발사항'으로 공시하는 반면, 지방자치단체 회계기준은 '우발손실'..."
96,사용에 혼란이 있고 분류상 오류가 있는 등 체계적 관리 대안에 대한 논의가 필요한 ...,"우발부채란 무엇이며, 그 관리가 왜 중요한가?",\n우발부채는 미래의 다양한 의제의무를 포괄하는 재정위험을 발생시키는 구체적 원인으...,,"우발부채는 미래의 다양한 의제의무를 포괄하는 재정위험을 발생시키는 구체적 원인으로,..."


In [74]:
if eval_mode == 'valid' :
  display("결과 : ",rslt_df["F1_0"].mean())
  display("결과 : ",rslt_df["F1_1"].mean())

In [75]:
if eval_mode == 'valid':
  # 제출용 샘플 파일 로드
  eval_df = rslt_df
  save_dir = os.path.join(path,'eval')
  if not os.path.exists(save_dir) : os.makedirs(save_dir)
  save_name = f'eval_{fname}'
  save_path = os.path.join(save_dir,save_name)

  # 결과를 CSV 파일로 저장
  eval_df.to_csv(save_path, encoding='UTF-8-sig', index=False)

In [76]:
#검증 데이터 쓸때만 사용 가능
if eval_mode == 'valid':
  for item in results:
      y_hat = item["Answer"]
      y = item["True_Answer"]
      f1, precision, recall = compute_f1(y, y_hat)
      item["F1"] = f1
      item["Precision"] = precision
      item["Recall"] = recall

# Submission

In [77]:
submit_df = pd.read_csv(f"{path}sample_submission.csv")
submit_df.head()

,SAMPLE_ID,Answer
0,TEST_000,데이콘
1,TEST_001,데이콘
2,TEST_002,데이콘
3,TEST_003,데이콘
4,TEST_004,데이콘


In [78]:
fname

'gemma2_large_ensemble_markdown_256_3epoch_refactored.csv'

In [79]:
fname = 'gemma2_large_ensemble_markdown_256_6epoch_refactored.csv'

In [80]:
# 제출용 샘플 파일 로드
submit_df = pd.read_csv(f"{path}sample_submission.csv")

# 생성된 답변을 제출 DataFrame에 추가
save_mode = 'submission'

if save_mode != 'submission' :
  submit_df['Question'] = [item['Question'] for item in results]
  submit_df['Context'] = [item['Context'] for item in results]
  save_dir = os.path.join(path,'eval')
else : save_dir = os.path.join(path,'sub')

if not os.path.exists(save_dir) : os.makedirs(save_dir)
save_path1 = os.path.join(save_dir,fname[:-4]+'_1.csv')
save_path2 = os.path.join(save_dir,fname[:-4]+'_2.csv')

submit_df['Answer'] = [item['Answer'] for item in results]
submit_df['Answer'] = submit_df['Answer'].fillna("데이콘").apply(str.strip)

# 결과를 CSV 파일로 저장
submit_df.to_csv(save_path1, encoding='UTF-8-sig', index=False)

# 후처리
#submit_df['Answer'] = submit_df.Answer.str.split('\n').str[0]
submit_df['Answer'] = submit_df.Answer.apply(post_process)
display(submit_df)
submit_df.to_csv(save_path2, encoding='UTF-8-sig', index=False)

,SAMPLE_ID,Answer
0,TEST_000,"2022년 혁신창업사업화자금(융자)의 예산은 2,330,000,000원입니다."
1,TEST_001,중소벤처기업부의 혁신창업사업화자금(융자) 사업목적은 중소기업이 보유한 우수 기술의 ...
2,TEST_002,"중소기업진흥에 관한 법률 제66조, 제67조, 제74조, 중소기업창업지원법 제35조..."
3,TEST_003,2010년에 재창업자금(실패 경영인에 대한 재기지원)이 신규 지원되었습니다.
4,TEST_004,2020년에는 미래기술육성자금과 고성장촉진자금이 신규로 지원되었습니다.
...,...,...
93,TEST_093,공적보증채무는 보증인이 기타 공공부문과 민간부문 단위 기존 채무의 원리금 상환을 보...
94,TEST_094,미래사회보장급여에 대한 순의무는 현행법과 규정에 따라 이미 가입자가 획득한 미래 급...
95,TEST_095,"국가결산보고서는 '우발사항'으로 공시하는 반면, 지방자치단체 회계기준은 '우발손실'..."
96,TEST_096,"우발부채는 미래의 다양한 의제의무를 포괄하는 재정위험을 발생시키는 구체적 원인으로,..."
